# How to query All of Us data in R?

This Notebook is a collection of best practices for writing big queries to extract All of Us (AoU) data based on frequently asked questions by users during office hours or from the User Support Hub. 

* How to query the visit_occurrence table

* How to extract large data

* How to run bash commands

## Setup

In [ ]:
library(bigrquery)
library(tidyverse) #imports stringr, dplyr and tidyr

In [ ]:
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
my_bucket

In [ ]:
DATASET <- Sys.getenv('WORKSPACE_CDR')
DATASET

In [ ]:
download_data <- function(query) {
  tb <- bq_project_query(Sys.getenv('GOOGLE_PROJECT'), query = str_glue(query)
                         , default_dataset = Sys.getenv('WORKSPACE_CDR'))
  bq_table_download(tb)
}

# How to query visit_occurrence table

We provided several examples demonstrating how to query the visit_occurrence table by writing custom queries.

**Example 1: What are the most common concepts in the visit_occurrence table?**

In [ ]:
query="
SELECT visit_concept_id, concept_name, COUNT(DISTINCT person_id) countp
FROM {DATASET}.visit_occurrence
JOIN {DATASET}.concept ON concept_id=visit_concept_id
GROUP BY 1,2
ORDER BY countp DESC
LIMIT 10
"
df = download_data(query)
dim(df)

In [ ]:
df

In [ ]:
# Save the data frame to a CSV file
write.csv(df, file = "df.csv", row.names = FALSE)

## How to extract data from the visit_occurrence table after a cohort is created by using Cohort Builder

**Example 2**: We start with this concept (Inpatient Visit, 9201) using Cohort Builder and have a query from the tool as shown in the cell below.

In [ ]:
dataset_32954956_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN (9201) 
                    AND is_standard = 1 )) criteria ) )", sep="")

We need to add {DATASET} to each OMOP table by running the cell below in order to run the query

In [ ]:
# library(stringr)
# Function to Add Dataset Name to SQL
inject_dataset <- function(sql_query, dataset_name) {
  # Regex pattern to match table names within backticks
  pattern <- "`(\\w+)`"
  
  # Add dataset name inside backticks
  sql_with_dataset <- str_replace_all(sql_query, pattern, paste0("`", dataset_name, ".\\1`"))
  
  return(sql_with_dataset)
}

# Example dataset name
dataset <- DATASET

# Apply function
query <- inject_dataset(dataset_32954956_person_sql, dataset)

# Print to verify
cat(query)

Run the query and check the dimension of the data frame.

In [ ]:
person_df = download_data(query)
dim(person_df)

From the query above, we only get the person_df. We will run the cell below to get actual data from the visit_occurrence table.

In [ ]:
query="
SELECT DISTINCT person_id, visit_concept_id, concept_name, 
FROM {DATASET}.visit_occurrence
JOIN {DATASET}.concept ON concept_id=visit_concept_id
WHERE visit_concept_id IN (9201)
"
visit_df = download_data(query)
dim(visit_df)

In [ ]:
length(unique(visit_df$person_id))

In [ ]:
head(visit_df)

**Example 3: We have a cohort created by combining both visit_occurrence and condition_occurrence tables**

We will use the concept (Inpatient Visit, 9201)  and lung cancer (443388) to create this cohort.

In [ ]:
# This query represents dataset "inpatient_lung_cancer_cb" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_36069044_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN (9201) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (443388)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )", sep="")

In [ ]:
# Apply function
query <- inject_dataset(dataset_36069044_person_sql, dataset)
# Print to verify
cat(query)

In [ ]:
# run the query 
person_df = download_data(query)
dim(person_df)

We need to run the cell below to get actual data from both tables.

In [ ]:
query="
SELECT DISTINCT co.person_id, 
condition_concept_id, condition_start_date, c1.concept_name,visit_concept_id,c2.concept_name,
visit_start_date,visit_end_date,
FROM {DATASET}.condition_occurrence co
JOIN {DATASET}.visit_occurrence v using (visit_occurrence_id)
JOIN {DATASET}.concept c1 ON condition_concept_id=c1.concept_id
join {DATASET}.concept_ancestor on descendant_concept_id=condition_concept_id
JOIN {DATASET}.concept c2 ON visit_concept_id=c2.concept_id
WHERE ancestor_concept_id IN (443388)
AND visit_concept_id IN (9201)
AND co.visit_occurrence_id IS NOT NULL"

In [ ]:
# run the query
visit_co_df = download_data(query)
dim(visit_co_df)

In [ ]:
# not everyone has actual data
length(unique(visit_co_df$person_id))

In [ ]:
head(visit_co_df)

# How to extract large data in R?

As seen here, https://bigrquery.r-dbi.org/reference/bq_table_download.html, the bq_table_download funtion from the BigQuery package is most suitable for results of smaller queries (<100 MB). For larger queries, we provide two additional methods demonstrating how to extract large data (i.e., 1M rows).

As of August 13, 2025, the cell below ran successfully. 

In [ ]:
query=str_glue("
SELECT
DISTINCT procedure.person_id,
p_standard_concept.concept_name as procedure,
procedure.procedure_datetime
FROM {DATASET}.procedure_occurrence procedure 
LEFT JOIN {DATASET}.concept p_standard_concept 
ON procedure.procedure_concept_id = p_standard_concept.concept_id
LIMIT 1000000")


In [ ]:
# will have errors for 1M rows, 50K rows is fine
df <- download_data(query)
dim(df)

**Method 1: to use python API**

Please restart the kernel before running the cell below. 

In [ ]:
library(tidyverse)
DATASET <- Sys.getenv('WORKSPACE_CDR')
query=str_glue("
SELECT
DISTINCT procedure.person_id,
p_standard_concept.concept_name as procedure,
procedure.procedure_datetime
FROM {DATASET}.procedure_occurrence procedure 
LEFT JOIN {DATASET}.concept p_standard_concept 
ON procedure.procedure_concept_id = p_standard_concept.concept_id
LIMIT 1000000")

In [ ]:
#temporary python setup
library(reticulate) 
use_python("/opt/conda/bin/python") 
# library(tidyverse)
bigquery=import("google.cloud.bigquery")

client = bigquery$Client()
DATASET =Sys.getenv('WORKSPACE_CDR')

job_config = bigquery$QueryJobConfig()
job_config$default_dataset =DATASET
query_job = client$query(query, job_config=job_config)  # API request

df= query_job$to_dataframe()
dim(df)

**Method 2: to use DBI interface and fetch data in chunks**

In [ ]:
# library(DBI)
# library(bigrquery)

# Set up the BigQuery connection
con <- dbConnect(
  bigrquery::bigquery(),
  project = Sys.getenv('GOOGLE_PROJECT'),
  dataset = DATASET
)

# Query with the DBI interface
# query <- "SELECT * FROM `your_dataset.your_table` WHERE ..."
res <- dbSendQuery(con, query)

# Fetch data in chunks
df_list <- list()
while(!dbHasCompleted(res)) {
  chunk <- dbFetch(res, n = 1e6) # Fetch 1 million rows at a time
  df_list <- append(df_list, list(chunk))
}

# Combine all chunks into a single data frame
df <- do.call(rbind, df_list)

# Clear the result and disconnect
dbClearResult(res)
dbDisconnect(con)

dim(df)

# How to run bash commands in R

Please see this notebook `01_2.How_to_work_with_env_variables_in_R.ipynb` for more examples.

You can use snippets for this purpose as well.

In [ ]:
# check files in the current working directory
system(paste0("ls *.csv "), intern=T)

In [ ]:
# save this df.csv to the bucket/data
system(paste0("gsutil cp *.csv ", my_bucket, "/data/"), intern=T)

In [ ]:
# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)

In [ ]:
# Copy the file from the bucket to the current working directory, with a different file name
system(paste0("gsutil cp ", my_bucket, "/data/df.csv", " df2.csv"), intern=T)

# Copy the file from the bucket to the current working directory, with same file name
# system(paste0("gsutil cp ", my_bucket, "/data/df.csv", " ."), intern=T)

In [ ]:
# check files in the current working directory
system(paste0("ls *.csv "), intern=T)